# Not Scaled Data

In [22]:
import pandas as pd
xls = pd.ExcelFile("Фрустрация.xlsx")
data = pd.read_excel(xls, xls.sheet_names)    

In [23]:
keys = [' E', ' E1', ' e_', ' M', ' M1', ' m_', ' I', ' I1', ' i_']

keys1 = [[' E', ' E1', ' e_'],
         [' M', ' M1', ' m_'],
         [' I', ' I1', ' i_']]

keys2 = [[' E', ' M', ' I'],
        [' E1', ' M1', ' I1'],
        [' e_', ' m_', ' i_']]
def get_y(keys, data, init_keys = keys):
    y = []
    for ind, i in enumerate(init_keys):
        for ind1, j in enumerate(keys):
            if i in j:
                y.append(ind1 * np.ones(len(data[i])))
    return np.hstack(y)

In [24]:
col = data[keys[0]].columns

In [25]:
def extract(list_of_texts):
    res = []
    for s in list_of_texts:
        k = 0
        for c in s:
            if c.islower():
                k += 1
        res.append(k/len(s))
    
    return pd.DataFrame({"Доля":res})

In [26]:
import json
with open("p3_texts.json", "r") as f:
    texts = f.read()
    texts = json.loads(texts)
    f.close()
print(texts.keys())
for key in keys:
    if key[-1] == '1':
        key1 = key[:-1] + "'"
    else:
        key1 = key
    data[key] = data[key].join( extract(texts['P3:' + key1]), rsuffix = "___")

dict_keys(['P3: E', 'P3: M', 'P3: m_', 'P3: e_', "P3: I'", "P3: E'", 'P3: i_', "P3: M'", 'P3: инф', 'P3: I', 'P3: n ("надо")'])


In [27]:
import numpy as np
from sklearn.preprocessing import scale
X = np.vstack([data[i].values for i in keys])
X = X[:, 1:]
y1 = get_y(keys1, data)
y2 = get_y(keys2, data)

In [28]:
print("Число слов", X[:, 2].mean())

Число слов 36.49063670411985


In [29]:
print("Число предложений", X[:, 6].mean())

Число предложений 3.4569288389513106


In [30]:
print("Число символов", X[:, 3].mean())

Число символов 188.49812734082397


In [31]:
# X = scale(X)
# X_bin = np.where(X <= 0, 0, 1)
X.shape

(267, 243)

## The First Union

In [93]:
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

from sklearn.feature_selection import SelectFromModel

from sklearn.linear_model import LogisticRegression, Lasso

model = Pipeline([
    #('first_scaler', StandardScaler()),
    #('first_select', SelectFromModel(LogisticRegression(C = 0.5, 
    #                                                    penalty="l1", 
    #                                                    multi_class = 'ovr', 
    #                                                    solver = 'liblinear'))),
    #('PolyPreproc', PolynomialFeatures(2)),
    ('second_scaler', StandardScaler()),
    ('second_select', SelectFromModel(LogisticRegression(C = 0.2, 
                                                        penalty="l1", 
                                                        multi_class = 'ovr', 
                                                        class_weight = "balanced",
                                                        solver = 'liblinear'))),
    ('main_model', LogisticRegression(C = 0.05,
                                      penalty = "l2",
                                      solver = "liblinear",
                                      class_weight = "balanced",
                                      multi_class = "ovr",
                                      max_iter = 100))
])

In [94]:
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle

s = 0
N = 250
for i in range(N):
    X_, y = shuffle(X, y1)
    s+= cross_val_score(model, X_, y, cv = 4, scoring = "f1_macro").mean()
s/N

/home/elias/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/elias/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/elias/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.3891043177465283

# Deep Learning Part

In [121]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
lb.fit(y1)
y = lb.transform(y1)

In [125]:
import torch
from torch import nn
from torch.functional import F
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [167]:
class Model(nn.Module):
    def __init__(self, n_feat):
        super(Model, self).__init__()
        self.model = []
        self.model.append(nn.Linear(in_features=n_feat, out_features=100))
        self.model.append(nn.Dropout())
        self.model.append(nn.ReLU())
        self.model.append(nn.BatchNorm1d(100))
        self.model.append(nn.Linear(in_features=100, out_features=1000))
        self.model.append(nn.Dropout())
        self.model.append(nn.ReLU())
        self.model.append(nn.BatchNorm1d(1000))
        self.model.append(nn.Linear(in_features=1000, out_features=3))
        self.model.append(nn.Sigmoid())
        self.model.append(nn.Softmax(dim = 1))
        [self.add_module(name = "final_layer%d"%(idx), module = layer) for idx, layer in enumerate(self.model)]
        

    def forward(self, whole_input):
        out = whole_input
        for layer in self.model:
            out = layer(out)
        
        return out

In [168]:
import time
history = []
s = 0
N = 10
t = time.time()
for k in range(N):
    X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size = 30)
    model = Model(61)
    class_weight = [len([j for j in y_train if j[i] == 1]) for i in range(3)]
    class_weight = [1/j for j in class_weight]
    opt = torch.optim.Adam(model.parameters(), lr=1e-4)
    history = []
    for i in range(2400):
        ix = np.random.randint(0, len(X_train), 40) 
        x_batch = torch.tensor(X_train[ix], dtype=torch.float32)
        y_batch = torch.tensor(y_train[ix], dtype=torch.float32)

        y_predicted = model(x_batch)

        loss = F.binary_cross_entropy(y_predicted, y_batch, weight = torch.tensor(class_weight, dtype=torch.float32))

        loss.backward()

        opt.step()

        opt.zero_grad()

        history.append(loss.data.numpy())
    x = torch.tensor(X_test, dtype=torch.float32)
    
    y_pred = model(x).data.numpy()
    s += f1_score(y_pred.argmax(axis = 1), y_test.argmax(axis=1), average = 'macro')
    print(f1_score(y_pred.argmax(axis = 1), y_test.argmax(axis=1), average = 'macro'))
    if k%3 == 0:
        print(k, (time.time() - t)/(k+1), s/(k+1))
s / N

0.3619047619047619
0 40.10628414154053 0.3619047619047619
0.3274853801169591
0.7302325581395349
0.39898989898989895
3 42.03406137228012 0.45465314978778876
0.45652173913043476
0.3387755102040817
0.6563011456628478
6 44.90193149021694 0.46717299916407423
0.27450980392156865
0.3878787878787879
0.4259259259259259
9 46.257529211044314 0.4358525511874801


0.4358525511874801